# Mod 3, Lsn 2 #
# Combining datasets #

Now that you know how to clean up your dataset, it's now time to combine them to make a larger data set.  If this is your first time building a dataset, then you need to think long and hard about what you want your final dataset to 'look like'.  What columns are you going to need, and in what order?  If you've already got a baseline dataset, and you're adding new data, you will be adding to the bottom of the dataset.  

Thinking about this, there are multiple ways to combine data sets into a single DataFrame.  Both _numpy_ and _pandas_ both concatenate - let's start with _numpy_.

First, let's create some variables to concatenate.

__Setup__

In [5]:
# Setup
import numpy as np
import pandas as pd  
x = [1,2,3]
y = [4,5,6]
z = [7,8,9]
np.concatenate([x,y,z])


array([1, 2, 3, 4, 5, 6, 7, 8, 9])

The first argument is the variables to concatenate.  The default _axis_ is zero, but it's not bad to define it everytime.


In [6]:
# Example 1
np.concatenate([x,y,z],axis=0)

array([1, 2, 3, 4, 5, 6, 7, 8, 9])

 Now let's try adding to the right. 

In [7]:
# Example 2
x = [[1,2],[3,4]]
np.concatenate([x,x],axis=1)

array([[1, 2, 1, 2],
       [3, 4, 3, 4]])

Notice I did two unique actions here - I created a single variable, and concatenated it with itself, to the right.

The pandas package provides various methods for combining DataFrames, including _append_, merge/_join_ and _concat_.

Let's start with the one with the lowest number of keystrokes, but not the smartest - _append_.


In [8]:
# Example 3
z.append(y)
print(z)

[7, 8, 9, [4, 5, 6]]


_append_ adds elements to the end of a list. _append_ is not the smartest way to modify a list - it is not an efficient method, as it involves multiple indices and a data buffer.  The smartest way to do multiples of these is to build a list of clean DataFrames and pass them to the _concat_ function covered later.

***

Now let's get fancy!

## Data ##

Let's use the Portal Teaching data, a subset of the data from Ernst et al's Long-term monitoring and experimental manipulation of a Chihuahuan ecosystem near Portal, Arizona.  

This database includes the species and weight of animals caught in sites at the study area.  The dataset is stored as a .csv file:  Each row holds info for a single animal and the columns are:

| Column    |   Description                   |
|:---------| :-------------------------------|
| record_id |  Unique id for the observation  |
| month     | month of observation            |
| day  |   day of observation                   |
| year |  year of observation  |
| plot_id     | ID of a particular site           |
| species_id | 2 letter code for a species |
| sex |  sex of animal ('M','F') |
| hindfoot_length | length of the hindfood in mm |
| weight | weight of the animal in grams |

In [9]:
# Example 4
surveys_df = pd.read_csv("surveys.csv")
surveys_df.info()
display(surveys_df.head(10))    # view the first 10 rows of data

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35549 entries, 0 to 35548
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   record_id        35549 non-null  int64  
 1   month            35549 non-null  int64  
 2   day              35549 non-null  int64  
 3   year             35549 non-null  int64  
 4   plot_id          35549 non-null  int64  
 5   species_id       34786 non-null  object 
 6   sex              33038 non-null  object 
 7   hindfoot_length  31438 non-null  float64
 8   weight           32283 non-null  float64
dtypes: float64(2), int64(5), object(2)
memory usage: 2.4+ MB


,record_id,month,day,year,plot_id,species_id,sex,hindfoot_length,weight
0,1,7,16,1977,2,NL,M,32.0,NaN
1,2,7,16,1977,3,NL,M,33.0,NaN
2,3,7,16,1977,2,DM,F,37.0,NaN
3,4,7,16,1977,7,DM,M,36.0,NaN
4,5,7,16,1977,3,DM,M,35.0,NaN
5,6,7,16,1977,1,PF,M,14.0,NaN
6,7,7,16,1977,2,PE,F,NaN,NaN
7,8,7,16,1977,1,DM,M,37.0,NaN
8,9,7,16,1977,1,DM,F,34.0,NaN
9,10,7,16,1977,6,PF,F,20.0,NaN


We can see that there were 35,549 rows parsed. Each row has 9 columns. The first column is the index of the DataFrame. The index is used to identify the position of the data, but it is not an actual column of the DataFrame.  Let's look at the type of each of the columns.

In [10]:
# Example 5
surveys_df.dtypes

record_id            int64
month                int64
day                  int64
year                 int64
plot_id              int64
species_id          object
sex                 object
hindfoot_length    float64
weight             float64
dtype: object

These files are large!  Let's work with something smaller, so we can tell what is going on..

We're going to use 'concat' function, in pandas, to concatenate the data, either at the bottom or the side.
If you've noticed, there is a single column added at the first - that is an 'index' added to each line to give it a unique number.  If we're going to add rows or columns, we need to make sure Python has it arranged correctly.  Sometimes we need to reset it, and sometimes we can ignore it.

In [11]:
# Example 6
# Read in the first ten lines of the surveys table
survey_sub = surveys_df.head(10)
# Now the bottom 10 lines
survey_sub_last10 = surveys_df.tail(10)
# Reset the index values to the second dataframe so that it appends correctly
survey_sub_last10 = survey_sub_last10.reset_index(drop=True)
# drop=True option avoids adding a new index column with old index values

Now we need to decide whether to add to the bottom or the side (right).  _axis=0_ tells pandas to stack the second UNDER the first.  It will automatically detect whether the column names are the same and will stack accordingly.  _axis=1_ will stack the columns to the RIGHT of the first.  To stack vertically, we need to make sure we have the same columns and associated column format in both datasets.  When we stack horizontally, we want to make sure what we are doing makes sense (i.e. the data are related in some way).

In [12]:
# Example 7
# Stack the DataFrames on top of each other
vertical_stack = pd.concat([survey_sub, survey_sub_last10], axis = 0)
# Stack the DataFrames side by side
horizontal_stack = pd.concat([survey_sub, survey_sub_last10], axis=1)

Now look at the two variables.  Notice anything unusual?

In [13]:
# Example 8
vertical_stack

,record_id,month,day,year,plot_id,species_id,sex,hindfoot_length,weight
0,1,7,16,1977,2,NL,M,32.0,NaN
1,2,7,16,1977,3,NL,M,33.0,NaN
2,3,7,16,1977,2,DM,F,37.0,NaN
3,4,7,16,1977,7,DM,M,36.0,NaN
4,5,7,16,1977,3,DM,M,35.0,NaN
5,6,7,16,1977,1,PF,M,14.0,NaN
6,7,7,16,1977,2,PE,F,NaN,NaN
7,8,7,16,1977,1,DM,M,37.0,NaN
8,9,7,16,1977,1,DM,F,34.0,NaN
9,10,7,16,1977,6,PF,F,20.0,NaN


In [14]:
# Example 8
horizontal_stack


,record_id,month,day,year,plot_id,species_id,sex,hindfoot_length,weight,record_id,month,day,year,plot_id,species_id,sex,hindfoot_length,weight
0,1,7,16,1977,2,NL,M,32.0,NaN,35540,12,31,2002,15,PB,F,26.0,23.0
1,2,7,16,1977,3,NL,M,33.0,NaN,35541,12,31,2002,15,PB,F,24.0,31.0
2,3,7,16,1977,2,DM,F,37.0,NaN,35542,12,31,2002,15,PB,F,26.0,29.0
3,4,7,16,1977,7,DM,M,36.0,NaN,35543,12,31,2002,15,PB,F,27.0,34.0
4,5,7,16,1977,3,DM,M,35.0,NaN,35544,12,31,2002,15,US,NaN,NaN,NaN
5,6,7,16,1977,1,PF,M,14.0,NaN,35545,12,31,2002,15,AH,NaN,NaN,NaN
6,7,7,16,1977,2,PE,F,NaN,NaN,35546,12,31,2002,15,AH,NaN,NaN,NaN
7,8,7,16,1977,1,DM,M,37.0,NaN,35547,12,31,2002,10,RM,F,15.0,14.0
8,9,7,16,1977,1,DM,F,34.0,NaN,35548,12,31,2002,7,DO,M,36.0,51.0
9,10,7,16,1977,6,PF,F,20.0,NaN,35549,12,31,2002,5,NaN,NaN,NaN,NaN


Notice that the index of _vertical_stack_ is duplicated!  Not good!  We need to reindex it to clean it up.

In [15]:
# Example 9
vertical_stack = vertical_stack.reset_index(drop=True)
vertical_stack

,record_id,month,day,year,plot_id,species_id,sex,hindfoot_length,weight
0,1,7,16,1977,2,NL,M,32.0,NaN
1,2,7,16,1977,3,NL,M,33.0,NaN
2,3,7,16,1977,2,DM,F,37.0,NaN
3,4,7,16,1977,7,DM,M,36.0,NaN
4,5,7,16,1977,3,DM,M,35.0,NaN
5,6,7,16,1977,1,PF,M,14.0,NaN
6,7,7,16,1977,2,PE,F,NaN,NaN
7,8,7,16,1977,1,DM,M,37.0,NaN
8,9,7,16,1977,1,DM,F,34.0,NaN
9,10,7,16,1977,6,PF,F,20.0,NaN


Now that I've gotten it cleaned up, I want to save it!  Save frequently!

I will want to save it to a .csv file, and without the index.  When the data will be (re)loaded it will create an index, so I don't want ANOTHER index..

The generic structure would be vertical_stack.to_csv('foldername/out.csv');


In [16]:
# Example 10
# write to csv
vertical_stack.to_csv('vertical_data.csv', index=False)

### Exercise 1 ###

There are two survey data files:  surveys2001.csv and surveys2002.csv.  Read the data into Python and combine the files to make a new variable.  Create a plot of average plot weight by year, grouped by sex.  Export your results as a CSV and make sure it reads back into Python properly.

----

This was an easy introduction.. Everything was identical.  Now, let's look at some examples that take a little more effort, but work for those that are more realistic.

We can use/pick a column that contains common values, which becomes a common unique id.  Combining DataFrames using a common field is called “joining”. The columns containing the common values are called “join key(s)”. Joining DataFrames in this way is often useful when one DataFrame is a “lookup table” containing additional data that we want to include in the other.

For example, the species.csv file that we’ve been working with is a lookup table. This table contains the genus, species and taxa code for 55 species. The species code is unique for each line. These species are identified in our survey data as well using the unique species code. Rather than adding 3 more columns for the genus, species and taxa to each of the 35,549 line Survey data table, we can maintain the shorter table with the species information. When we want to access that information, we can create a query that joins the additional columns of information to the Survey data.

Storing data in this way has many benefits including:
<ol>
<li> It ensures consistency in the spelling of species attributes (genus, species and taxa) given each species is only entered once. Imagine the possibilities for spelling errors when entering the genus and species thousands of times! </li>
<li> It also makes it easy for us to make changes to the species information once without having to find each instance of it in the larger survey data.  </li>
<li> It optimizes the size of our data. </li>
</ol>

Let's practice.

In [17]:
# Example 11
# Read in first 10 lines of surveys table
survey_sub = surveys_df.head(10)

# Import a small subset of the species data designed for this part of the lesson.
species_sub = pd.read_csv('speciesSubset.csv', keep_default_na=False, na_values=[""])

In this example, species_sub is the lookup table containing genus, species, and taxa names that we want to join with the data in survey_sub to produce a new DataFrame that contains all of the columns from both species_df and survey_df.

## Identifying join keys ##
To identify appropriate join keys we first need to know which field(s) are shared between the files (DataFrames). We might inspect both DataFrames to identify these columns. If we are lucky, both DataFrames will have columns with the same name that also contain the same data. If we are less lucky, we need to identify a (differently-named) column in each DataFrame that contains the same information.


In [18]:
# Example 12
species_sub.columns

Index(['species_id', 'genus', 'species', 'taxa'], dtype='object')

In [19]:
# Example 13
survey_sub.columns

Index(['record_id', 'month', 'day', 'year', 'plot_id', 'species_id', 'sex',
       'hindfoot_length', 'weight'],
      dtype='object')

We know that these are from the same, so the columns are the same.  Imagine they were different, and so we would end up choosing _species_id_ as the 'join' key.

Now that we've decided on the fields to use as a 'join', we are almost ready.  Guess what, there are different types of 'joins'!  We will need to decide which type of join to use.

## Inner join ##
The most common type of join is called an _inner join_.  An inner join combines two DataFrames based on a join key and returns a DataFrame that contains **only** those rows that have matching values in both of the original DataFrames.

![Inner join](inner-join.png)



In [20]:
# Example 14
merged_inner = pd.merge(left=survey_sub, right=species_sub, left_on='species_id', right_on='species_id')
# In this case `species_id` is the only column name in  both dataframes, so if we skipped `left_on`
# And `right_on` arguments we would still get the same result

# What's the size of the output data?
merged_inner.shape
merged_inner


,record_id,month,day,year,plot_id,species_id,sex,hindfoot_length,weight,genus,species,taxa
0,1,7,16,1977,2,NL,M,32.0,NaN,Neotoma,albigula,Rodent
1,2,7,16,1977,3,NL,M,33.0,NaN,Neotoma,albigula,Rodent
2,3,7,16,1977,2,DM,F,37.0,NaN,Dipodomys,merriami,Rodent
3,4,7,16,1977,7,DM,M,36.0,NaN,Dipodomys,merriami,Rodent
4,5,7,16,1977,3,DM,M,35.0,NaN,Dipodomys,merriami,Rodent
5,8,7,16,1977,1,DM,M,37.0,NaN,Dipodomys,merriami,Rodent
6,9,7,16,1977,1,DM,F,34.0,NaN,Dipodomys,merriami,Rodent
7,7,7,16,1977,2,PE,F,NaN,NaN,Peromyscus,eremicus,Rodent


The result of an inner join of survey_sub and species_sub is a new DataFrame that contains the combined set of columns from survey_sub and species_sub. It only contains rows that have two-letter species codes that are the same in both the survey_sub and species_sub DataFrames. In other words, if a row in survey_sub has a value of species_id that does not appear in the species_id column of species, it will not be included in the DataFrame returned by an inner join. Similarly, if a row in species_sub has a value of species_id that does not appear in the species_id column of survey_sub, that row will not be included in the DataFrame returned by an inner join.

The two DataFrames that we want to join are passed to the merge function using the left and right argument. The left_on='species' argument tells merge to use the species_id column as the join key from survey_sub (the left DataFrame). Similarly , the right_on='species_id' argument tells merge to use the species_id column as the join key from species_sub (the right DataFrame). For inner joins, the order of the left and right arguments does not matter.

The result merged_inner DataFrame contains all of the columns from survey_sub (record id, month, day, etc.) as well as all the columns from species_sub (species_id, genus, species, and taxa).

Notice that merged_inner has fewer rows than survey_sub. This is an indication that there were rows in surveys_df with value(s) for species_id that do not exist as value(s) for species_id in species_df.

## Left joins ##
What if we want to add information from species_sub to survey_sub without losing any of the information from survey_sub? In this case, we use a different type of join called a “left outer join”, or a “left join”.

Like an inner join, a left join uses join keys to combine two DataFrames. Unlike an inner join, a left join will return all of the rows from the left DataFrame, even those rows whose join key(s) do not have values in the right DataFrame. Rows in the left DataFrame that are missing values for the join key(s) in the right DataFrame will simply have null (i.e., NaN or None) values for those columns in the resulting joined DataFrame.

![Left join](left-join.png)

Note: a left join will still discard rows from the right DataFrame that do not have values for the join key(s) in the left DataFrame.

In [21]:
# Example 15
merged_left = pd.merge(left=survey_sub, right=species_sub, how='left', left_on='species_id', right_on='species_id')
merged_left

,record_id,month,day,year,plot_id,species_id,sex,hindfoot_length,weight,genus,species,taxa
0,1,7,16,1977,2,NL,M,32.0,NaN,Neotoma,albigula,Rodent
1,2,7,16,1977,3,NL,M,33.0,NaN,Neotoma,albigula,Rodent
2,3,7,16,1977,2,DM,F,37.0,NaN,Dipodomys,merriami,Rodent
3,4,7,16,1977,7,DM,M,36.0,NaN,Dipodomys,merriami,Rodent
4,5,7,16,1977,3,DM,M,35.0,NaN,Dipodomys,merriami,Rodent
5,6,7,16,1977,1,PF,M,14.0,NaN,NaN,NaN,NaN
6,7,7,16,1977,2,PE,F,NaN,NaN,Peromyscus,eremicus,Rodent
7,8,7,16,1977,1,DM,M,37.0,NaN,Dipodomys,merriami,Rodent
8,9,7,16,1977,1,DM,F,34.0,NaN,Dipodomys,merriami,Rodent
9,10,7,16,1977,6,PF,F,20.0,NaN,NaN,NaN,NaN


The result DataFrame from a left join (merged_left) looks very much like the result DataFrame from an inner join (merged_inner) in terms of the columns it contains. However, unlike merged_inner, merged_left contains the same number of rows as the original survey_sub DataFrame. When we inspect merged_left, we find there are rows where the information that should have come from species_sub (i.e., species_id, genus, and taxa) is missing (they contain NaN values):



In [22]:
# Example 16
merged_left[ pd.isnull(merged_left.genus) ]

,record_id,month,day,year,plot_id,species_id,sex,hindfoot_length,weight,genus,species,taxa
5,6,7,16,1977,1,PF,M,14.0,NaN,NaN,NaN,NaN
9,10,7,16,1977,6,PF,F,20.0,NaN,NaN,NaN,NaN


These rows are the ones where the value of species_id from survey_sub (in this case, PF) does not occur in species_sub.

## Other join types ##
The pandas merge function supports two other join types:

<ul>
<li>Right (outer) join: Invoked by passing how='right' as an argument. Similar to a left join, except all rows from the right DataFrame are kept, while rows from the left DataFrame without matching join key(s) values are discarded. </li>
<li>Full (outer) join: Invoked by passing how='outer' as an argument. This join type returns the all pairwise combinations of rows from both DataFrames; i.e., the result DataFrame will NaN where data is missing in one of the dataframes. This join type is very rarely used. </li>
    
</ul>

---
### Exercise 2 ###

1.  The plots.csv file contains information about the type associated with each plot.  Use that data to summarize the number of plots by plot type.

2.  Calculate a diversity index of your choice for control vs rodent exclosure plots. The index should consider both species abundance and number of species. You might choose to use the simple biodiversity index described here which calculates diversity as:

the number of species in the plot / the total number of individuals in the plot = Biodiversity index.